## Загрузка библиотек

In [1]:
import tarfile
import re

from sklearn.model_selection import train_test_split

from collections import Counter, defaultdict
import random

## 1: Препроцессинг

Загружаем датасет [French-English, 292 short sentences](https://www.cis.lmu.de/~fraser/EMA2008/model1.html)
(для этого надо нажать на "German-English, 292 short sentences", т.к. на сайте перепутаны ссылки на скачивание)

In [78]:
#извлекаем файлы из архива
with tarfile.open('fr-en.tiny.tgz', 'r:gz') as tar:
  tar.extractall()

Вызовем команду shell `ls`, чтобы увидеть, что мы распаковали.

In [79]:
!ls

fr-en.tiny.en                  fr-en.tiny.tgz
fr-en.tiny.fr                  ДЗ3_Фадеева_smt.ipynb


Создаем 2 выборки и токенизируем тексты по фразам.

In [80]:
with open('fr-en.tiny.en', 'r') as f:
  english = f.read().split('\n')[:-1]

with open('fr-en.tiny.fr', 'r') as f:
  french = f.read().split('\n')[:-1]

def cleaning_(lang):
    #топорный метод, но в "белый лист" добавляем только буквы
    whitelist = set('abcdefghijklmnopqrstuvwxyz ABCDEFGHIJKLMNOPQRSTUVWXYZabcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZàâäçèéêëîïôœùûüÿÀÂÄÇÈÉÊËÎÏÔŒÙÛÜŸ')
    result_list = []
    for sentence in lang:
        #приводим всё к нижнему регистру, оставляем только буквы из "белого листа"
        result_list.append((''.join(filter(whitelist.__contains__, sentence))).lower())
    return result_list

english = cleaning_(english)
french = cleaning_(french)

print("Данные языка X:\n", english[:7]) #выведем 7 предложений
print("Данные языка Y:\n", french[:7])

Данные языка X:
 ['resumption of the session', 'please rise  then  for this minute  s silence ', ' the house rose and observed a minute  s silence ', 'madam president  on a point of order ', 'if the house agrees  i shall do as mr evans has suggested ', 'madam president  on a point of order ', 'i would like your advice about rule  concerning inadmissibility ']
Данные языка Y:
 ['reprise de la session', 'je vous invite à vous lever pour cette minute de silence ', ' le parlement  debout  observe une minute de silence ', 'madame la présidente  c est une motion de procédure ', 'si l assemblée en est d accord  je ferai comme m evans l a suggéré ', 'madame la présidente  c est une motion de procédure ', 'je voudrais vous demander un conseil au sujet de l article   qui concerne l irrecevabilité ']


Разделим нашу выборку с помощью sklearn.

In [81]:
X_train, X_test, y_train, y_test = train_test_split(english, french)

cn = 0
print("> Обучающая выборка:")
for text, label in zip(X_train, y_train):
    print(f"\nТекст на французском: {label}\n Его перевод на английский: {text}\n")
    cn+=1
    if cn==3:
        break #чтобы выводились не все предложения, а только часть

cn = 0
print("> Тестовая выборка:")
for text, label in zip(X_test, y_test):
    print(f"\nТекст на французском: {label}\n Его перевод на английский: {text}\n")
    cn+=1
    if cn==3:
        break

> Обучающая выборка:

Текст на французском: je suis d accord avec votre analyse 
 Его перевод на английский: i agree with your analysis 


Текст на французском: je voudrais vous demander un conseil au sujet de l article   qui concerne l irrecevabilité 
 Его перевод на английский: i would like your advice about rule  concerning inadmissibility 


Текст на французском: mon groupe soutiendra d ailleurs toute initiative visant à améliorer la sécurité des transports 
 Его перевод на английский: when it comes to safety my group will always support any initiatives to improve transport safety 

> Тестовая выборка:

Текст на французском: la plupart des propositions d  amendement sont de nature purement technique 
 Его перевод на английский: most of the proposed amendments are of a purely technical nature 


Текст на французском: les subventions ne suffisent pas pour faire du développement  quand font défaut les infrastructures ou les services publics 
 Его перевод на английский: subsidies are n

### Подготовка данных

Проведем пословную токенизацию каждой фразы.

In [82]:
def tokenize(sentences):
  #функция возвращает списки слов
  return [sentence.split() for sentence in sentences]

X_train_tokens, X_test_tokens, y_train_tokens, y_test_tokens = tokenize(X_train), tokenize(X_test), tokenize(y_train), tokenize(y_test)

print('Образец токенизированного текста:', X_train_tokens[:10])

Образец токенизированного текста: [['i', 'agree', 'with', 'your', 'analysis'], ['i', 'would', 'like', 'your', 'advice', 'about', 'rule', 'concerning', 'inadmissibility'], ['when', 'it', 'comes', 'to', 'safety', 'my', 'group', 'will', 'always', 'support', 'any', 'initiatives', 'to', 'improve', 'transport', 'safety'], ['the', 'market', 'favours', 'the', 'short', 'term', 'and', 'immediate', 'profits'], ['our', 'group', 'is', 'in', 'favour', 'of', 'adopting', 'this', 'report'], ['the', 'report', 'put', 'forward', 'today', 'reestablishes', 'its', 'place', 'in', 'the', 'political', 'sphere'], ['mr', 'wynn', 'that', 'makes', 'sense'], ['they', 'are', 'supported', 'by', 'appropriate', 'european', 'union', 'structural', 'fund', 'activities'], ['the', 'european', 'commission', 's', 'sixth', 'report', 'presents', 'very', 'valuable', 'conclusions'], ['this', 'report', 'is', 'very', 'good', 'and', 'our', 'group', 'supports', 'it']]


Создадим словарь уникальных словоформ.

In [83]:
x_vocab = Counter(' '.join(french).split()).keys()
y_vocab = Counter(' '.join(english).split()).keys()

print(f"Словарь французских словоформ: {list(x_vocab)[:50]}...\n Всего {len(x_vocab)} словоформ")
print(f"\nCловарь английских словоформ: {list(y_vocab)[:50]}...\n Всего {len(y_vocab)} словоформ")

Словарь французских словоформ: ['reprise', 'de', 'la', 'session', 'je', 'vous', 'invite', 'à', 'lever', 'pour', 'cette', 'minute', 'silence', 'le', 'parlement', 'debout', 'observe', 'une', 'madame', 'présidente', 'c', 'est', 'motion', 'procédure', 'si', 'l', 'assemblée', 'en', 'd', 'accord', 'ferai', 'comme', 'm', 'evans', 'a', 'suggéré', 'voudrais', 'demander', 'un', 'conseil', 'au', 'sujet', 'article', 'qui', 'concerne', 'irrecevabilité', 'il', 'précise', 'que', 'cela']...
 Всего 1081 словоформ

Cловарь английских словоформ: ['resumption', 'of', 'the', 'session', 'please', 'rise', 'then', 'for', 'this', 'minute', 's', 'silence', 'house', 'rose', 'and', 'observed', 'a', 'madam', 'president', 'on', 'point', 'order', 'if', 'agrees', 'i', 'shall', 'do', 'as', 'mr', 'evans', 'has', 'suggested', 'would', 'like', 'your', 'advice', 'about', 'rule', 'concerning', 'inadmissibility', 'it', 'says', 'that', 'should', 'be', 'done', 'despite', 'principle', 'relative', 'stability']...
 Всего 978 сло

## 2: Модель SMT

### IBM 1 Expectation-Maximization (t-model)

In [84]:
#вероятность того, что случайное слово x_vocab соответсвует случайному слову y_vocab
uniform = 1 / (len(x_vocab) * len(y_vocab))

f'{uniform:.10f}'

'0.0000009459'

In [85]:
#t-model
t = {}

for i in range(len(X_train)):
  #начинаем итерацию по обучающей выборке
  for word_x in X_train_tokens[i]:
    for word_y in y_train_tokens[i]:
      #создаем t-table
      t[(word_x, word_y)] = uniform

#t-table
cn = 0
print ("T-table")
for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))
  cn+=1
  if cn==10:
    break #чтобы вывести не всё-всё, а только часть

T-table
Соответствие | i   ->   je | Вероятность: 0.0
Соответствие | i   ->   suis | Вероятность: 0.0
Соответствие | i   ->   d | Вероятность: 0.0
Соответствие | i   ->   accord | Вероятность: 0.0
Соответствие | i   ->   avec | Вероятность: 0.0
Соответствие | i   ->   votre | Вероятность: 0.0
Соответствие | i   ->   analyse | Вероятность: 0.0
Соответствие | agree   ->   je | Вероятность: 0.0
Соответствие | agree   ->   suis | Вероятность: 0.0
Соответствие | agree   ->   d | Вероятность: 0.0


In [138]:
#количество итераций обучения
epochs = 18

In [154]:
for epoch in range(epochs):
  #начинаем обучение

  #шаг 0. создаем слоты для подсчета статистики
  count = {} # P(x|y)
  total = {} # P(y)

  for i in range(len(X_train)):
    #начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        #создаем слоты для подсчета условной вероятности совпадений в корпусе
        count[(word_x, word_y)] = 0
        #и слоты для статистической языковой модели y
        total[word_y] = 0

  #шаг 1. Expectation
  for i in range(len(X_train)):
    #начинаем итерацию по обучающей выборке
    total_stat = {} # статистика x

    #собираем предварительную статистику на основе данных x
    for word_x in X_train_tokens[i]:
      total_stat[word_x] = 0 # создаем слоты для подсчета статистики по каждому токену x
      for word_y in y_train_tokens[i]:
        #обновляем данные из t-table; увеличиваем значения при обнаружении совместной встречаемости
        total_stat[word_x] += t[(word_x, word_y)]

    #обновляем данные для P(x|y) и P(y)
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        #подсчет условной вероятности совпадений в корпусе: равномерное распределение / частотность x
        count[(word_x, word_y)] += t[(word_x, word_y)] / total_stat[word_x]
        #подсчет статистической информации y: равномерное распределение / частотность x
        total[word_y] += t[(word_x, word_y)] / total_stat[word_x]

  #шаг 2. Maximization
  for i in range(len(X_train)):
    #начинаем итерацию по обучающей выборке
    for word_x in X_train_tokens[i]:
      for word_y in y_train_tokens[i]:
        #обновляем t-table: вероятность совпадения в корпусе / вероятность информации y
        t[(word_x, word_y)] = count[(word_x, word_y)] / total[word_y]

cn = 0
print ("Соответствие, вероятность")
for elem in t:
  print("Соответствие |", elem[0], "  ->  ", elem[1], "| Вероятность:", round(t[elem], 3))
  cn+=1
  if cn==30:
    break #чтобы вывести не всё-всё, а только часть

Соответствие, вероятность
Соответствие | i   ->   je | Вероятность: 1.0
Соответствие | i   ->   suis | Вероятность: 0.0
Соответствие | i   ->   d | Вероятность: 0.0
Соответствие | i   ->   accord | Вероятность: 0.254
Соответствие | i   ->   avec | Вероятность: 0.0
Соответствие | i   ->   votre | Вероятность: 0.0
Соответствие | i   ->   analyse | Вероятность: 0.0
Соответствие | agree   ->   je | Вероятность: 0.0
Соответствие | agree   ->   suis | Вероятность: 0.359
Соответствие | agree   ->   d | Вероятность: 0.0
Соответствие | agree   ->   accord | Вероятность: 0.268
Соответствие | agree   ->   avec | Вероятность: 0.0
Соответствие | agree   ->   votre | Вероятность: 0.0
Соответствие | agree   ->   analyse | Вероятность: 0.0
Соответствие | with   ->   je | Вероятность: 0.0
Соответствие | with   ->   suis | Вероятность: 0.282
Соответствие | with   ->   d | Вероятность: 0.0
Соответствие | with   ->   accord | Вероятность: 0.21
Соответствие | with   ->   avec | Вероятность: 0.135
Соответст

Получается правдоподобно, например:

Соответствие | i   ->   je | Вероятность: 1.0

Соответствие | your   ->   votre | Вероятность: 0.69

### Биграммная модель

In [155]:
#для обучения модели объединим 2 выборки
tokens = ' '.join(french).split()

#хранилище для биграмм
bigram_model = defaultdict(list)

#собираем все попарные совпадения
for i in range(len(tokens)-1):
    current_word = tokens[i]
    next_word = tokens[i + 1]
    bigram_model[current_word].append(next_word)

print(list(bigram_model)[:10])

def decoder(model, steps=5):
  #инициализация случайного токена
  current_word = random.choice(tokens)
  generated_sentence = current_word

  for step in range(steps):
    #пошаговая генерация
    print('Шаг', step+1)
    next_word_options = model[current_word]
    print(f'Правдоподобные варианты продолжения для токена {current_word}:', next_word_options)

    current_word = random.choice(next_word_options)
    generated_sentence += ' '
    generated_sentence += current_word
    print('Промежуточный результат:', generated_sentence)
    print()
  print('Результат:', generated_sentence)

decoder(bigram_model)

['reprise', 'de', 'la', 'session', 'je', 'vous', 'invite', 'à', 'lever', 'pour']
Шаг 1
Правдоподобные варианты продолжения для токена les: ['escaliers', 'zones', 'choses', 'états', 'producteurs', 'décisions', 'modifications', 'positions', 'explications', 'positions', 'normes', 'intervenants', 'gens', 'régions', 'états', 'quatre', 'riches', 'pauvres', 'projets', 'conséquences', 'patrons', 'capacités', 'pme', 'projets', 'fonds', 'subventions', 'infrastructures', 'services', 'états', 'fonds', 'fonds', 'écarts', 'efforts', 'résultats', 'situations', 'régions', 'régions', 'fruits', 'choses', 'rapports', 'données', 'états', 'états', 'règles', 'lignes', 'gouvernements', 'subventions', 'pays', 'monopoles', 'monopoles', 'prix', 'régions', 'gains', 'aides', 'aides', 'travailleurs']
Промежуточный результат: les patrons

Шаг 2
Правдоподобные варианты продолжения для токена patrons: ['des']
Промежуточный результат: les patrons des

Шаг 3
Правдоподобные варианты продолжения для токена des: ['princip

## 3: Оценка результатов

In [156]:
#сортировка t-table по убыванию правдоподобия
sorted_t = sorted(t.items(), key = lambda k:(k[1], k[0]), reverse = True)
#print(sorted_t)
def translate(token):
  for element in sorted_t:
    if element[0][1] == token:
      # поиск совпадений в t-table
      return element[0][0]

cn = 0
for sentence in y_test_tokens:
  print("Оригинальное предложение:", ' '.join(sentence))
  translation = []
  for token in sentence:
    if translate(token):
        translation.append(translate(token))
  print("Перевод:", ' '.join(translation))
  cn+=1
  if cn==20:
     break #чтобы вывести 20 шт.

Оригинальное предложение: la plupart des propositions d amendement sont de nature purement technique
Перевод: the the very to very who of
Оригинальное предложение: les subventions ne suffisent pas pour faire du développement quand font défaut les infrastructures ou les services publics
Перевод: the one not not to a the development inspire transparent the obstacle will the will turned
Оригинальное предложение: ce point est d une extrême importance
Перевод: this point is to a
Оригинальное предложение: nous obtenons progressivement secteur après secteur que les états membres appliquent des règles minimales communes
Перевод: we were were that the states member the rules minimum
Оригинальное предложение: je suggère à m evans d aller relire le règlement
Перевод: i to mr suggested to together the regulation
Оригинальное предложение: ce n est pas demander beaucoup
Перевод: this not is not your very
Оригинальное предложение: cela constituera un test pour ce qui est de la coopération judicieuse 

In [157]:
from nltk.translate.bleu_score import corpus_bleu

print(y_test_tokens[0])
candidate = [translate(token) for token in y_test_tokens[0] if translate(token)]
reference = [X_test_tokens[i] for i in range(len(candidate))]

print('reference:    ',reference)
print('candidate:     ',candidate)
print(len(reference), len(candidate))
bleu_score = corpus_bleu(reference, candidate)

print("BLEU Score:", bleu_score)

['la', 'plupart', 'des', 'propositions', 'd', 'amendement', 'sont', 'de', 'nature', 'purement', 'technique']
reference:     [['most', 'of', 'the', 'proposed', 'amendments', 'are', 'of', 'a', 'purely', 'technical', 'nature'], ['subsidies', 'are', 'not', 'enough', 'to', 'ensure', 'development', 'when', 'infrastructure', 'and', 'public', 'services', 'are', 'lacking'], ['this', 'latter', 'point', 'is', 'of', 'particular', 'importance'], ['in', 'area', 'after', 'area', 'we', 'are', 'now', 'obtaining', 'common', 'minimum', 'regulations', 'for', 'the', 'member', 'states'], ['i', 'suggest', 'mr', 'evans', 'goes', 'back', 'and', 'reads', 'the', 'regulation'], ['it', 'is', 'not', 'a', 'lot', 'to', 'ask'], ['this', 'will', 'be', 'a', 'good', 'test', 'as', 'to', 'whether', 'there', 'is', 'reasonable', 'cooperation', 'between', 'the', 'two', 'institutions']]
candidate:      ['the', 'the', 'very', 'to', 'very', 'who', 'of']
7 7
BLEU Score: 4.739433789388674e-78
